<a href="https://colab.research.google.com/github/alexisdr/uned-tfg-deteccion-eas/blob/main/UNED-TFG-5-metrica-s-test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Metricas S

Calcual el valor de las métricas S mediante los datos de test

# Parámetros
* ruta_base: ruta en la que se encuentran los datos del corpus
* ruta_dataset: ruta en el que se almacenará el dataset
* usar_subconjunto_datos_codigos_mas_frecuentes: se usarán los actos clínicos que implican los códigos más frecuentes

In [9]:
ruta_base = '/drive/My Drive/CorpusPFG/'

#Datasets procesados
ruta_dataset = ruta_base + 'Dataset-mas-frecuentes-con-none'
modelo_base = "alexisdr/uned-tfg-08.100_mas_frecuentes_con_none"
usar_subconjunto_datos_codigos_mas_frecuentes = True
umbral = 0.9

Instalación de depndencias necesarias

In [2]:
!pip install -q datasets transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.1 MB/s eta 0:00:00


# Carga de datos del dataset

In [3]:
from google.colab import drive

drive.mount('/drive')

Mounted at /drive


In [4]:
from datasets import DatasetDict

dataset = DatasetDict.load_from_disk(ruta_dataset)

dataset

DatasetDict({
    train: Dataset({
        features: ['acto', 'label', 'label_str', 'label_list', 'label_list_str', 'informes', 'text', 'json'],
        num_rows: 2218
    })
    validation: Dataset({
        features: ['acto', 'label', 'label_str', 'label_list', 'label_list_str', 'informes', 'text', 'json'],
        num_rows: 247
    })
    test: Dataset({
        features: ['acto', 'label', 'label_str', 'label_list', 'label_list_str', 'informes', 'text', 'json'],
        num_rows: 3425
    })
    trainMasFrecuentes: Dataset({
        features: ['acto', 'label', 'label_str', 'label_list', 'label_list_str', 'informes', 'text', 'json'],
        num_rows: 739
    })
    validationMasFrecuentes: Dataset({
        features: ['acto', 'label', 'label_str', 'label_list', 'label_list_str', 'informes', 'text', 'json'],
        num_rows: 83
    })
    testMasFrecuentes: Dataset({
        features: ['acto', 'label', 'label_str', 'label_list', 'label_list_str', 'informes', 'text', 'json'],
       

In [5]:
train = "train"
validation = "validation"
test = "test"

if (usar_subconjunto_datos_codigos_mas_frecuentes):
  train = "trainMasFrecuentes"
  validation = "validationMasFrecuentes"
  test = "testMasFrecuentes"

In [6]:
from datasets import ClassLabel

class2label = dataset[train].features["label"]
print(class2label)

ClassLabel(names=['NONE', 'T38.0X5A', 'T45.515A', 'T50.2X5A', 'Y95', 'P01.1', 'T81.4XXA', 'T45.1X5A'], id=None)


In [7]:
class2label.num_classes

8

# Preprocesado de los datos

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(modelo_base)

def preprocess_data(examples):
  return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors="pt")

# Cálculo de la métrica S

In [11]:
#Calcula el prefico comun entre 2 códigos
def calculo_lcs(codigo_i, codigo_j):
  if codigo_i is None or codigo_j is None:
    return ""

  #Se omiten los puntos existenten en los códigos
  codigo_i = codigo_i.replace(".", "")
  codigo_j = codigo_j.replace(".", "")

  #Tomamos el tamaño mínimo
  size = min(len(codigo_i), len(codigo_j))

  lcs_i_j = ''
  for i in range(size):
    if codigo_i[i] == codigo_j[i]:
      lcs_i_j = lcs_i_j + codigo_i[i]
    else:
      return lcs_i_j

  return lcs_i_j


#Devuelve la longitud de la cadena de caracteres C
#si esta vale al menos 3, y devuelve 0 si dicha longitud es menor que 3.
#Esto se debe a que los códigos CIE-10 más cortos contienen al menos 3 caracteres.
def calculo_ic(codigo):
  #Se omiten los puntos existenten en los códigos
  codigo = codigo.replace(".", "")

  tamanyo = len(codigo)
  if (tamanyo < 3):
    return 0
  else:
    return tamanyo

#similitud entre 2 códigos CIE-10 𝑖 y 𝑗:
def calculo_c(codigo_i, codigo_j):
  divisor = calculo_ic(codigo_i) + calculo_ic(codigo_j)
  if (divisor == 0):
    return 0
  dividendo = 2 * calculo_ic(calculo_lcs(codigo_i, codigo_j))
  c_i_j = dividendo / divisor
  return round(c_i_j, 6)

def metrica_s(lista_codigos_i, lista_codigos_j):
  #las listas deben tener valores
  if (len(lista_codigos_i) == 0 or len(lista_codigos_j) == 0):
    return 0

  #max (Ng, Ns)
  divisor = max(len(lista_codigos_i), len(lista_codigos_j))
  if (divisor == 0):
    return 0

  max_c_i_j = 0
  for codigo_j in lista_codigos_j:
    max_local_c_i_j = 0
    for codigo_i in lista_codigos_i:
      c_i_j = calculo_c(codigo_i, codigo_j)
      if (c_i_j > max_local_c_i_j):
        max_local_c_i_j = c_i_j
    max_c_i_j += max_local_c_i_j

  s = max_c_i_j / divisor
  return round(s, 6)

def metrica_s_train (y_true, y_pred):
    y_true_labels = []
    true_labels = [class2label.int2str([idx])for idx, label in enumerate(y_true) if label == 1.0]
    for label in true_labels:
      y_true_labels.append(label[0])

    y_pred_labels = []
    pred_labels = [class2label.int2str([idx])for idx, label in enumerate(y_pred) if label == 1.0]
    for label in pred_labels:
      y_pred_labels.append(label[0])

    return metrica_s(y_true_labels, y_pred_labels)

# Inferencia

Se ejecuta el modelo contra datos de test y se calcula la métrica S

In [12]:
from transformers import AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained(
    modelo_base,
    num_labels=class2label.num_classes,
    problem_type = "multi_label_classification")

In [13]:
codigo_NONE = 'NONE'

def calcular_predicciones(probs, threshold=umbral):
  predictions = np.zeros(probs.shape)
  predictions[np.where(probs > threshold)] = 1
  return predictions

def calcular_predicciones_NONE_mas_probable(probs, threshold=umbral):
  predictions = calcular_predicciones(probs, threshold)

  try:
    probabilidad_NONE = probs[class2label.str2int(codigo_NONE)]
    #Se ha encontrado NONE entre las predicciones
    if (probabilidad_NONE > 0):
      predictions_mayor_que_prob_NONE = np.zeros(probs.shape)
      predictions_mayor_que_prob_NONE[np.where(probs > probabilidad_NONE)] = 1
      # Si hay labels con mejor probabilidad que none nos la quedamos
      if (1 in predictions_mayor_que_prob_NONE):
        predictions = predictions_mayor_que_prob_NONE
      else: #Si no las hay nos quedamos solo con NONE
        predictions[:] = 0
        predictions[class2label.str2int(codigo_NONE)] = 1
  except ValueError:
    pass

  return predictions

#Solo asignar NONE si existe y tiene una probabilidad que supere el umbral
def calcular_predicciones_existe_NONE(probs, threshold=umbral):
  predictions = calcular_predicciones(probs, threshold)

  try:
    probabilidad_NONE = probs[class2label.str2int(codigo_NONE)]
    #Se ha encontrado NONE entre las predicciones, nos la quedamos de forma exclusiva
    if (probabilidad_NONE > threshold):
        predictions[:] = 0
        predictions[class2label.str2int(codigo_NONE)] = 1
  except ValueError:
    pass

  return predictions

In [14]:
# obtienen los nombres de las etiquetas predichas
def obtener_lables_predichas (y_pred):
  predicted_labels = [class2label.int2str([idx])for idx, label in enumerate(y_pred) if label == 1.0]
  y_pred_labels = []
  for label in predicted_labels:
      y_pred_labels.append(label[0])
  return y_pred_labels

# obtienen el vector y_true a partir de los labels
def obtener_y_true (labels):
  # crea una matriz del tamaño del texto y las clases a entrenar
  labels_matrix = np.zeros(class2label.num_classes)
  for clase in labels:
    try:
      indice_clase = class2label.str2int(clase)
      labels_matrix[indice_clase] = 1
    except:
      print("La clase %s no está entre las clases de entrenamiento" % clase)
  return labels_matrix.tolist()

In [15]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

df_resultados = pd.DataFrame(columns=["y_true", "y_pred", "metrica_s", "precision", "recall", "f1", "accuracy"])

def calculo_metricas_multi_label(y_pred, y_true, threshold=umbral):
    metric_average = "micro"
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    #ConfusionMatrixDisplay(cm).plot()
    #print("tn", tn, "tn", fp, "fn", fn, "tp", tp )

    #precision_score_value = precision_score(y_true, y_pred, average=metric_average)
    precision_score_value = tp / (tp + fp)
    #recall_score_value = recall_score(y_true, y_pred, average=metric_average)
    recall_score_value = tp / (tp + fn)
    #f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average=metric_average)
    #print("precision_score_value", precision_score_value, "recall_score_value", recall_score_value )
    f1_micro_average = 0
    if (tp > 0):
      f1_micro_average = 2 * ((precision_score_value*recall_score_value)/(precision_score_value+recall_score_value))
    #roc_auc = roc_auc_score(y_true, y_pred, average=metric_average)
    accuracy = accuracy_score(y_true, y_pred)
    metrica_s_value = metrica_s_train(y_true, y_pred)

    return metrica_s_value, precision_score_value, recall_score_value, f1_micro_average, accuracy

for i in tqdm(range(dataset[test].num_rows)):
  data = dataset[test][i]
  inputs = preprocess_data(data)
  outputs = model(**inputs)

  logits = outputs.logits
  # apply sigmoid + threshold
  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(logits.squeeze().cpu())

  y_pred = calcular_predicciones_NONE_mas_probable(probs)
  y_pred_labels = obtener_lables_predichas(y_pred)

  y_true = obtener_y_true(data['label_list_str'])
  y_true_labels = data['label_list_str']

  metricas = calculo_metricas_multi_label(y_pred, y_true)

  resultados = ({
      'y_true':y_true_labels,
      'y_pred' :y_pred_labels,
      'metrica_s':metricas[0],
      'precision':metricas[1],
      'recall':metricas[2],
      'f1':metricas[3],
      'accuracy':metricas[4]})

  df_resultados = df_resultados.append(resultados, ignore_index=True)


  0%|          | 0/3020 [00:00<?, ?it/s]

La clase E09.9 no está entre las clases de entrenamiento
La clase E16.0 no está entre las clases de entrenamiento
La clase T86.01 no está entre las clases de entrenamiento
La clase E09.9 no está entre las clases de entrenamiento
La clase E09.9 no está entre las clases de entrenamiento
La clase T46.0X5A no está entre las clases de entrenamiento
La clase E09.9 no está entre las clases de entrenamiento
La clase Y84.2 no está entre las clases de entrenamiento
La clase Y83.1 no está entre las clases de entrenamiento
La clase T82.897A no está entre las clases de entrenamiento
La clase T46.4X5A no está entre las clases de entrenamiento
La clase E89.2 no está entre las clases de entrenamiento
La clase T42.75XA no está entre las clases de entrenamiento
La clase T46.5X5A no está entre las clases de entrenamiento
La clase T50.0X5A no está entre las clases de entrenamiento
La clase E89.0 no está entre las clases de entrenamiento
La clase T84.52XA no está entre las clases de entrenamiento
La clase 

#Macro-average

La métrica S macro-average se calcula como la media aritmética de la métrica S sobre todos los actos clínicos del conjunto de pruebas

In [16]:
def calcular_media (metrica_nombre):
  suma = df_resultados[metrica_nombre].sum()
  media = round(suma / dataset[test].num_rows, 6)
  print(metrica_nombre, media)


calcular_media ('metrica_s')
calcular_media ('precision')
calcular_media ('recall')
calcular_media ('f1')
calcular_media ('accuracy')

metrica_s 0.009063
precision 0.008411
recall 0.04106
f1 0.013923
accuracy 0.260306


In [17]:
df_resultados

,y_true,y_pred,metrica_s,precision,recall,f1,accuracy
0,[NONE],"[T38.0X5A, T50.2X5A, P01.1, T81.4XXA, T45.1X5A]",0.0,0.0,0.0,0,0.25
1,[NONE],"[T38.0X5A, T50.2X5A, P01.1, T81.4XXA, T45.1X5A]",0.0,0.0,0.0,0,0.25
2,[NONE],"[T38.0X5A, T50.2X5A, P01.1, T81.4XXA, T45.1X5A]",0.0,0.0,0.0,0,0.25
3,[NONE],"[T38.0X5A, T50.2X5A, P01.1, T81.4XXA, T45.1X5A]",0.0,0.0,0.0,0,0.25
4,[NONE],"[T38.0X5A, T50.2X5A, P01.1, T81.4XXA, T45.1X5A]",0.0,0.0,0.0,0,0.25
...,...,...,...,...,...,...,...
3015,[T45.1X5A],"[T38.0X5A, T50.2X5A, P01.1, T81.4XXA, T45.1X5A]",0.2,0.2,1.0,0.333333,0.50
3016,[T45.1X5A],"[T38.0X5A, T50.2X5A, P01.1, T81.4XXA, T45.1X5A]",0.2,0.2,1.0,0.333333,0.50
3017,"[T45.1X5A, T40.2X5A]","[T38.0X5A, T50.2X5A, P01.1, T81.4XXA, T45.1X5A]",0.2,0.2,1.0,0.333333,0.50
3018,[T45.1X5A],"[T38.0X5A, T50.2X5A, P01.1, T81.4XXA, T45.1X5A]",0.2,0.2,1.0,0.333333,0.50


#Micro-average

El cálculo de la métrica S micro-average sobre un conjunto de códigos seleccionado se realiza de la siguiente manera: se calcula la medida S de cada código sobre todos los actos clínicos en los que aparece en el Gold Standard y posteriormente se divide por el número de códigos del conjunto de códigos que se quiere evaluar. Nótese que para ello, en la evaluación sólo se tiene en cuenta los códigos del conjunto, de manera que el resto deben obviarse.

In [21]:
class2label

ClassLabel(names=['NONE', 'T38.0X5A', 'T45.515A', 'T50.2X5A', 'Y95', 'P01.1', 'T81.4XXA', 'T45.1X5A'], id=None)

In [22]:
df_resultados_micro = pd.DataFrame(columns=["clase", "metrica_s_micro"])

for clase in class2label.names:
  codigo_busqueda = clase
  suma_metrica_s_micro = 0
  indice = 0
  for row in df_resultados['y_true']:
    for codigo in row:
      if (codigo == codigo_busqueda):
        suma_metrica_s_micro += df_resultados['metrica_s'][indice]
        break
    indice += 1
  micro_average = round(suma_metrica_s_micro / class2label.num_classes, 6)

  resultados = ({
      'clase':clase,
      'metrica_s_micro': micro_average})

  df_resultados_micro = df_resultados_micro.append(resultados, ignore_index=True)

def calcular_media_micro (metrica_nombre):
  suma = df_resultados_micro[metrica_nombre].sum()
  media = round(suma / class2label.num_classes, 6)
  print(metrica_nombre, media)

calcular_media_micro ('metrica_s_micro')

print(df_resultados_micro)

metrica_s_micro 0.449107
      clase  metrica_s_micro
0      NONE         0.000000
1  T38.0X5A         0.885714
2  T45.515A         0.296428
3  T50.2X5A         0.810714
4       Y95         0.050000
5     P01.1         0.600000
6  T81.4XXA         0.600000
7  T45.1X5A         0.350000
